In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

import ROOT as rt
import uproot
#from root_numpy import root2array, tree2array
#from root_pandas import read_root
import h5py 

from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import keras.backend as K
from keras.metrics import CategoricalAccuracy
from keras.losses import CategoricalCrossentropy

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

import numpy as np
import numpy.lib.recfunctions as nlr
import pandas as pd
import os, sys
from matplotlib import pyplot as plt
import math
try:
    import setGPU
except:
    os.environ['KERAS_BACKEND'] = 'tensorflow'
    os.environ['CUDA_VISIBLE_DEVICES'] = '5'

import gpustat
gpustat.print_gpustat()
os.environ['CUDA_VISIBLE_DEVICES']='3'

gpu-ibanks-3.hep.caltech.edu  Tue Aug 24 14:28:59 2021  470.57.02
[0] NVIDIA GeForce GTX 1080 | 24'C,   0 % |     1 /  8119 MB |
[1] NVIDIA GeForce GTX 1080 | 26'C,   0 % |     1 /  8119 MB |
[2] NVIDIA GeForce GTX 1080 | 27'C,   0 % |     1 /  8119 MB |
[3] NVIDIA GeForce GTX 1080 | 24'C,   0 % |     1 /  8119 MB |
[4] NVIDIA GeForce GTX 1080 | 27'C,   0 % |     1 /  8119 MB |
[5] NVIDIA GeForce GTX 1080 | 27'C,   0 % |   727 /  8119 MB | 3012(723M)
[6] NVIDIA GeForce GTX 1080 | 27'C,   0 % |     1 /  8119 MB |
[7] NVIDIA GeForce GTX 1080 | 28'C,   0 % |     1 /  8119 MB |


### Load MC samples

In [3]:
def samp_to_df(samp_name, total_num):
    dfs = []
    tree_vars = [b'leading_photon_pt', b'subleading_photon_pt', b'diphoton_pt', b'diphoton_mass',
           b'leading_bjet_pt', b'subleading_bjet_pt', b'dibjet_pt', b'dibjet_mass',
           b'leading_bjet_pt_corr', b'subleading_bjet_pt_corr', b'dibjet_pt_corr', b'dibjet_mass_corr',
           b'leading_pho_pt_over_dimass', b'leading_bjet_pt_over_dimass', b'leading_bjet_pt_over_dimass_corr',
           b'subleading_pho_pt_over_dimass', b'subleading_bjet_pt_over_dimass', b'subleading_bjet_pt_over_dimass_corr',
           b'leading_photon_phi', b'leading_photon_eta', b'subleading_photon_phi', b'subleading_photon_eta',
           b'diphoton_eta', b'photon_delR', b'diphoton_pt_over_diphoton_mass',
           b'leading_bjet_phi', b'leading_bjet_eta', b'subleading_bjet_phi', b'subleading_bjet_eta',
           b'dibjet_eta', b'bjet_delR', b'dibjet_pt_over_dibjet_mass', b'dibjet_pt_over_dibjet_mass_corr',
           b'leadingDeepBscore', b'subleadingDeepBscore', b'sumDeepBscore',
           b'rec_pho_bjet_min_dR', b'all_pho_bjet_min_dR', b'dphi_met_leading_bjet', b'dphi_met_subleading_bjet',
           b'leading_vbfjet_pt', b'subleading_vbfjet_pt', b'divbfjet_pt', b'divbfjet_mass',
           b'leading_vbfjet_phi', b'leading_vbfjet_eta', b'subleading_vbfjet_phi', b'subleading_vbfjet_eta',
           b'leading_vbfjet_pt_over_dimass', b'subleading_vbfjet_pt_over_dimass', b'divbfjet_pt_over_dimass',
           b'divbfjet_eta', b'vbfjet_delR', b'vbfjet_del_eta',
           b'MET_pt', b'MET_phi', b'MET_sumEt',
           b'recon', b'ggHH_recon', b'VBFHH_recon', b'genweight', b'nbjet', b'nvbfjet', b'nphoton', b'njet',
           b'run', b'lumi', b'event']
    for i in range(total_num+1):
        file_name = f'/storage/af/user/schen7/CMSSW_9_4_2/src/Higgs/HHbbgg/HHbbggAna/condor/output/{samp_name}{i}.root'
        samp_file = uproot.open(file_name)
        samp_array = samp_file['tree'].arrays(tree_vars)
        samp_df = pd.DataFrame(samp_array)
        dfs.append(samp_df)
    combine_df = pd.concat(dfs)
    return combine_df

In [4]:
# Set up dataframes - 2018 

# Signals
GluGluToHH_df_2018 = samp_to_df('job_2_ntuple20180819v1/GluGluToHHTo2B2G_node_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8Job2ifile', 0)
VBFHH_df_2018= samp_to_df('job_3_ntuple20180819v1/VBFHHTo2B2G_CV_1_C2V_1_C3_1_TuneCP5_PSWeights_13TeV-madgraph-pythia8Job3ifile', 0)

# Backgrounds
VHToGG_df_2018 = samp_to_df('job_4_ntuple20180819v1/VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8Job4ifile', 0)
ttHToGG_df_2018 = samp_to_df('job_5_ntuple20180819v1/ttHToGG_M125_TuneCP5_PSweights_13TeV-powheg-pythia8Job5ifile', 0)
VBFHToGG_df_2018 = samp_to_df('job_6_ntuple20180819v1/VBFHToGG_M125_13TeV_amcatnlo_pythia8Job6ifile', 0)
GluGluHToGG_df_2018 = samp_to_df('job_7_ntuple20180819v1/GluGluHToGG_M125_TuneCP5_13TeV-amcatnloFXFX-pythia8Job7ifile', 0)
TTJets_df_2018 = samp_to_df('job_8_ntuple20180819v1/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8Job8ifile', 60)
TTGJets_df_2018 = samp_to_df('job_9_ntuple20180819v1/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8Job9ifile', 3)
TTGG_0Jets_df_2018 = samp_to_df('job_10_ntuple20180819v1/TTGG_0Jets_TuneCP5_13TeV_amcatnlo_madspin_pythia8Job10ifile', 5)
GJet_SmallPt_df_2018 = samp_to_df('job_11_ntuple20180819v1/GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job11ifile', 3)
GJet_BigPt_df_2018 = samp_to_df('job_12_ntuple20180819v1/GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job12ifile', 2)
DiPhotonJetsBox_df_2018 = samp_to_df('job_13_ntuple20180819v1/DiPhotonJetsBox_MGG-80toInf_13TeV-SherpaJob13ifile', 4)
DiPhotonJetsBox1B_df_2018 = samp_to_df('job_14_ntuple20180819v1/DiPhotonJetsBox1BJet_MGG-80toInf_13TeV-SherpaJob14ifile', 0)
DiPhotonJetsBox2B_df_2018 = samp_to_df('job_15_ntuple20180819v1/DiPhotonJetsBox2BJets_MGG-80toInf_13TeV-SherpaJob15ifile', 0)
QCD_Jets_df_2018 = samp_to_df('job_16_ntuple20180819v1/QCD_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job16ifile', 15)

# Set up dataframes - 2017

# Signals
GluGluToHH_df_2017 = samp_to_df('job_2_ntuple20170819v1/GluGluToHHTo2B2G_node_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8Job2ifile', 3)
VBFHH_df_2017 = samp_to_df('job_3_ntuple20170819v1/VBFHHTo2B2G_CV_1_C2V_1_C3_1_13TeV-madgraphJob3ifile', 1)

# Backgrounds
VHToGG_df_2017 = samp_to_df('job_4_ntuple20170819v1/VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8Job4ifile', 0)
ttHToGG_df_2017 = samp_to_df('job_5_ntuple20170819v1/ttHToGG_M125_13TeV_powheg_pythia8Job5ifile', 0)
VBFHToGG_df_2017 = samp_to_df('job_6_ntuple20170819v1/VBFHToGG_M-125_13TeV_powheg_pythia8Job6ifile', 2)
GluGluHToGG_df_2017 = samp_to_df('job_7_ntuple20170819v1/GluGluHToGG_M-125_13TeV_powheg_pythia8Job7ifile', 0)
TTJets_df_2017 = samp_to_df('job_8_ntuple20170819v1/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8Job8ifile', 38)
TTGJets_df_2017 = samp_to_df('job_9_ntuple20170819v1/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8Job9ifile', 10)
TTGG_0Jets_df_2017 = samp_to_df('job_10_ntuple20170819v1/TTGG_0Jets_TuneCP5_13TeV_amcatnlo_madspin_pythia8Job10ifile', 1)
GJet_SmallPt_df_2017 = samp_to_df('job_11_ntuple20170819v1/GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job11ifile', 2)
GJet_BigPt_df_2017 = samp_to_df('job_12_ntuple20170819v1/GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job12ifile', 20)
DiPhotonJetsBox_df_2017 = samp_to_df('job_1_ntuple20170819v2/DiPhotonJetsBox_MGG-80toInf_13TeV-SherpaJob1ifile', 16)
DiPhotonJetsBox1B_df_2017 = samp_to_df('job_14_ntuple20170819v1/DiPhotonJetsBox1BJet_MGG-80toInf_13TeV-SherpaJob14ifile', 3)
DiPhotonJetsBox2B_df_2017 = samp_to_df('job_15_ntuple20170819v1/DiPhotonJetsBox2BJets_MGG-80toInf_13TeV-SherpaJob15ifile', 5)
QCD_Jets_df_2017 = samp_to_df('job_16_ntuple20170819v1/QCD_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCP5_13TeV_Pythia8Job16ifile', 3)

# Set up dataframes - 2016 

# Signals
GluGluToHH_df_2016 = samp_to_df('job_2_ntuple20160819v1/GluGluToHHTo2B2G_node_cHHH1_TuneCUETP8M1_PSWeights_13TeV-powheg-pythia8Job2ifile', 0)
VBFHH_df_2016 = samp_to_df('job_3_ntuple20160819v1/VBFHHTo2B2G_CV_1_C2V_1_C3_1_13TeV-madgraphJob3ifile', 0)

# Background
VHToGG_df_2016 = samp_to_df('job_4_ntuple20160819v1/VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8Job4ifile', 0)
ttHToGG_df_2016 = samp_to_df('job_5_ntuple20160819v1/ttHToGG_M125_13TeV_powheg_pythia8_v2Job5ifile', 0)
VBFHToGG_df_2016 = samp_to_df('job_6_ntuple20160819v1/VBFHToGG_M125_13TeV_amcatnlo_pythia8_v2Job6ifile', 2)
GluGluHToGG_df_2016 = samp_to_df('job_7_ntuple20160819v1/GluGluHToGG_M125_13TeV_amcatnloFXFX_pythiaJob7ifile', 0)
TTJets_df_2016 = samp_to_df('job_8_ntuple20160819v1/TTJets_TuneCUETP8M2T4_13TeV-amcatnloFXFX-pythia8Job8ifile', 8)
TTGJets_df_2016 = samp_to_df('job_9_ntuple20160819v1/TTGJets_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8Job9ifile', 2)
TTGG_0Jets_df_2016 = samp_to_df('job_10_ntuple20160819v1/TTGG_0Jets_TuneCUETP8M1_13TeV_amcatnlo_madspin_pythia8Job10ifile', 0)
GJet_SmallPt_df_2016 = samp_to_df('job_11_ntuple20160819v1/GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8Job11ifile', 2)
GJet_BigPt_df_2016 = samp_to_df('job_12_ntuple20160819v1/GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8Job12ifile', 12)
DiPhotonJetsBox_df_2016 = samp_to_df('job_13_ntuple20160819v1/DiPhotonJetsBox_MGG-80toInf_13TeV-SherpaJob13ifile', 9)
DiPhotonJetsBox1B_df_2016 = samp_to_df('job_14_ntuple20160819v1/DiPhotonJetsBox1BJet_MGG-80toInf_TuneSherpa_13TeV-SherpaJob14ifile', 3)
DiPhotonJetsBox2B_df_2016 = samp_to_df('job_15_ntuple20160819v1/DiPhotonJetsBox2BJets_MGG-80toInf_TuneSherpa_13TeV-SherpaJob15ifile', 4)
QCD_Jets_df_2016 = samp_to_df('job_16_ntuple20160819v1/QCD_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8Job16ifile', 3)

# Combine by sample (recon == 1; photon cut == 90)

# Signals 
GluGluToHH_df = pd.concat([GluGluToHH_df_2018, GluGluToHH_df_2017, GluGluToHH_df_2016], ignore_index=True)
VBFHH_df = pd.concat([VBFHH_df_2018, VBFHH_df_2017, VBFHH_df_2016], ignore_index=True)

# Backgrounds
VHToGG_df = pd.concat([VHToGG_df_2018, VHToGG_df_2017, VHToGG_df_2016], ignore_index=True)
ttHToGG_df = pd.concat([ttHToGG_df_2018, ttHToGG_df_2017, ttHToGG_df_2016], ignore_index=True)
VBFHToGG_df = pd.concat([VBFHToGG_df_2018, VBFHToGG_df_2017, VBFHToGG_df_2016], ignore_index=True)
GluGluHToGG_df = pd.concat([GluGluHToGG_df_2018, GluGluHToGG_df_2017, GluGluHToGG_df_2016], ignore_index=True)
TTJets_df= pd.concat([TTJets_df_2018, TTJets_df_2017, TTJets_df_2016], ignore_index=True)
TTGJets_df = pd.concat([TTGJets_df_2018, TTGJets_df_2017, TTGJets_df_2016], ignore_index=True)
TTGG_0Jets_df = pd.concat([TTGG_0Jets_df_2018, TTGG_0Jets_df_2017, TTGG_0Jets_df_2016], ignore_index=True)
GJet_SmallPt_df = pd.concat([GJet_SmallPt_df_2018, GJet_SmallPt_df_2017, GJet_SmallPt_df_2016], ignore_index=True)
GJet_BigPt_df = pd.concat([GJet_BigPt_df_2018, GJet_BigPt_df_2017, GJet_BigPt_df_2016], ignore_index=True)
DiPhotonJetsBox_df = pd.concat([DiPhotonJetsBox_df_2018, DiPhotonJetsBox_df_2017, DiPhotonJetsBox_df_2016], ignore_index=True)
DiPhotonJetsBox1B_df = pd.concat([DiPhotonJetsBox1B_df_2018, DiPhotonJetsBox1B_df_2017, DiPhotonJetsBox1B_df_2016], ignore_index=True)
DiPhotonJetsBox2B_df = pd.concat([DiPhotonJetsBox2B_df_2018, DiPhotonJetsBox2B_df_2017, DiPhotonJetsBox2B_df_2016], ignore_index=True)
QCD_Jets_df = pd.concat([QCD_Jets_df_2018, QCD_Jets_df_2017, QCD_Jets_df_2016], ignore_index=True)

In [5]:
# Select ggHH signal specifically -- pho == 90

dfs_combine = [GluGluToHH_df, VBFHH_df,
               VHToGG_df, ttHToGG_df, VBFHToGG_df, GluGluHToGG_df, 
                TTJets_df, TTGJets_df, TTGG_0Jets_df, 
            GJet_SmallPt_df, GJet_BigPt_df,
             DiPhotonJetsBox_df, DiPhotonJetsBox1B_df, DiPhotonJetsBox2B_df, QCD_Jets_df]

dfs_ggHH = []

for i in range(len(dfs_combine)):
    df_ggHH = dfs_combine[i][dfs_combine[i][b'ggHH_recon'] == 1]
    dfs_ggHH.append(df_ggHH)

## Preprocessing

In [6]:
mc_labels = ['GluGluToHH', 'VBFToHH',
            'VHToGG', 'ttHToGG', 'VBFHToGG', 'GluGluHtoGG',
            'TTJets', 'TTGJets', 'TTGG_0Jets',
            'GJet_SmallPt', 'GJet_BigPt', 'DiPhotonJetsBox', 'DiPhotonJetsBox1B', 'DiPhotonJetsBox2B', 'QCDJets']

dfs_ggHH_label = []

for i in range(len(mc_labels)):
    df = pd.DataFrame()
    for column in list(dfs_ggHH[i].columns):
        df[column] = dfs_ggHH[i].loc[:, column].values
    samp_label = [mc_labels[i]] * len(dfs_ggHH[i].index)
    df[b'label'] = samp_label
    samp_label_num = [i] * len(dfs_ggHH[i].index)
    df[b'label_num'] = samp_label_num
    dfs_ggHH_label.append(df)

full_df = pd.concat(dfs_ggHH_label, ignore_index=True)
full_df.head()

,b'leading_photon_pt',b'subleading_photon_pt',b'diphoton_pt',b'diphoton_mass',b'leading_bjet_pt',b'subleading_bjet_pt',b'dibjet_pt',b'dibjet_mass',b'leading_bjet_pt_corr',b'subleading_bjet_pt_corr',...,b'genweight',b'nbjet',b'nvbfjet',b'nphoton',b'njet',b'run',b'lumi',b'event',b'label',b'label_num'
0,81.548317,68.587204,84.036278,124.439453,80.1250,60.7500,88.046196,111.977684,89.827637,63.123047,...,-0.000028,3,0,2,6,1,1121,112017,GluGluToHH,0
1,109.037231,87.997322,153.144653,124.160385,115.9375,69.3125,160.948395,111.282120,117.749023,75.133667,...,0.000028,2,0,2,5,1,1121,112010,GluGluToHH,0
2,99.698898,54.716755,92.352104,124.160637,92.5625,26.7500,112.699577,106.316811,100.517090,32.967285,...,0.000028,2,1,2,10,1,1121,112001,GluGluToHH,0
3,78.673645,74.914726,111.698479,120.000145,178.5000,31.0625,172.392776,125.170486,164.816162,27.756042,...,0.000028,2,0,2,13,1,1121,112006,GluGluToHH,0
4,447.419708,159.403656,594.103027,123.621674,485.0000,46.3750,525.561584,98.438232,465.817871,57.153564,...,0.000028,2,0,2,8,1,1121,112019,GluGluToHH,0


In [28]:
# define variables
training_vars = [b'leading_photon_eta', b'leading_photon_phi', 
                b'subleading_photon_eta', b'subleading_photon_phi',
                 b'leading_bjet_pt_corr', b'leading_bjet_eta', b'leading_bjet_phi',
                 b'subleading_bjet_pt_corr', b'subleading_bjet_eta', b'subleading_bjet_phi',
                 b'leadingDeepBscore', b'subleadingDeepBscore', b'sumDeepBscore',
                 b'leading_pho_pt_over_dimass', b'subleading_pho_pt_over_dimass',
                 b'diphoton_pt_over_diphoton_mass', b'dibjet_pt_over_dibjet_mass_corr',
                 b'rec_pho_bjet_min_dR', b'all_pho_bjet_min_dR', b'dphi_met_leading_bjet', b'dphi_met_subleading_bjet',
                 b'MET_pt', b'MET_phi', b'MET_sumEt',
                 b'nbjet', b'nphoton', b'njet'
                ]

aug_vars = [b'dibjet_mass', b'diphoton_mass', b'event']

w_var = [b'genweight']

# select df columns
x_df = full_df[training_vars]
x_full_df = full_df[training_vars + aug_vars + w_var]
y_df = full_df[b'label_num']
X = x_df.values
Y = y_df.values

In [29]:
# split into training and test; following: https://www.kaggle.com/hungdo1291/keras-dnn-multi-class
random_seed=2
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=random_seed)
Y_train_dummy = np_utils.to_categorical(Y_train)

## Model Setup

In [21]:
# # following: https://www.kaggle.com/hungdo1291/keras-dnn-multi-class

# def precision(y_true, y_pred): 
#     """Precision metric. Only computes a batch-wise average of precision.  
# -    Computes the precision, a metric for multi-label classification of 
# -    how many selected items are relevant. 
# -    """ 
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1))) 
#     precision = true_positives / (predicted_positives + K.epsilon()) 
#     return precision 


# def recall(y_true, y_pred): 
#     """Recall metric. 
# -    Only computes a batch-wise average of recall. 
# -    Computes the recall, a metric for multi-label classification of 
# -    how many relevant items are selected. 
# -    """ 
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1))) 
#     recall = true_positives / (possible_positives + K.epsilon()) 
#     return recall 


# def fbeta_score(y_true, y_pred, beta=1): 

#     """Computes the F score.  
# -    The F score is the weighted harmonic mean of precision and recall. 
# -    Here it is only computed as a batch-wise average, not globally. 
# -    This is useful for multi-label classification, where input samples can be 
# -    classified as sets of labels. By only using accuracy (precision) a model 
# -    would achieve a perfect score by simply assigning every class to every 
# -    input. In order to avoid this, a metric should penalize incorrect class 
# -    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0) 
# -    computes this, as a weighted mean of the proportion of correct class 
# -    assignments vs. the proportion of incorrect class assignments.  
# -    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning 
# -    correct classes becomes more important, and with beta > 1 the metric is 
# -    instead weighted towards penalizing incorrect class assignments. 
# -    """ 
#     if beta < 0: 
#         raise ValueError('The lowest choosable beta is zero (only precision).') 

#     # If there are no true positives, fix the F score at 0 like sklearn. 
#     if K.sum(K.round(K.clip(y_true, 0, 1))) == 0: 

#         return 0 
#     p = precision(y_true, y_pred) 
#     r = recall(y_true, y_pred) 
#     bb = beta ** 2 
#     fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon()) 
#     return fbeta_score 

# def fmeasure(y_true, y_pred): 
#     """Computes the f-measure, the harmonic mean of precision and recall. 
#     Here it is only computed as a batch-wise average, not globally. 
#     """ 
#     return fbeta_score(y_true, y_pred, beta=1) 

In [10]:
def multi_DNN(n_hidden = 1, first_neuron = 100, n_neurons = 64, dropout_rate = 0.4, activation_func = 'sigmoid', lr= 0.00002):
    model = Sequential()
    model.add(Dense(first_neuron, input_dim=len(training_vars), activation='relu'))
    model.add(Dropout(dropout_rate))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation = activation_func))
        model.add(Dropout(dropout_rate))
    model.add(Dense(15, activation = 'softmax'))
    opt = Adam(learning_rate = lr)
    model.compile(loss=CategoricalCrossentropy(), optimizer=opt, metrics=[CategoricalAccuracy()])
    return model

## Model Test

In [32]:
model = multi_DNN()
history = model.fit(X_train, Y_train_dummy, epochs=200,batch_size=64, validation_split = 0.2, 
            callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')])

Epoch 1/200
33026/33026 [==============================] - 57s 2ms/step - loss: 1.7864 - categorical_accuracy: 0.5749 - val_loss: 1.6291 - val_categorical_accuracy: 0.5966
Epoch 2/200
33026/33026 [==============================] - 57s 2ms/step - loss: 1.6649 - categorical_accuracy: 0.5972 - val_loss: 1.6123 - val_categorical_accuracy: 0.5966
Epoch 3/200
33026/33026 [==============================] - 56s 2ms/step - loss: 1.6359 - categorical_accuracy: 0.5982 - val_loss: 1.5924 - val_categorical_accuracy: 0.6001
Epoch 4/200
33026/33026 [==============================] - 57s 2ms/step - loss: 1.6112 - categorical_accuracy: 0.5992 - val_loss: 1.5519 - val_categorical_accuracy: 0.6001
Epoch 5/200
33026/33026 [==============================] - 56s 2ms/step - loss: 1.5757 - categorical_accuracy: 0.6000 - val_loss: 1.5158 - val_categorical_accuracy: 0.6003
Epoch 6/200
33026/33026 [==============================] - 56s 2ms/step - loss: 1.5400 - categorical_accuracy: 0.6040 - val_loss: 1.4966 - v

In [33]:
Y_val_dummy = np_utils.to_categorical(Y_val)
model.evaluate(X_val, Y_val_dummy)

20641/20641 [==============================] - 24s 1ms/step - loss: 1.1360 - categorical_accuracy: 0.6739


[1.1360400915145874, 0.6738505363464355]

In [19]:
output = model.predict(X_val)

In [21]:
print(output[1])

[8.0512249e-01 6.7714551e-03 3.1499397e-03 5.6102205e-02 4.2781346e-03
 5.7956960e-02 2.2415115e-04 9.1528433e-05 7.5978220e-05 2.1229168e-04
 1.6477642e-03 3.1100949e-02 1.0379279e-03 3.2087151e-02 1.4109601e-04]


In [34]:
compare = pd.DataFrame()
output = model.predict(X_val)
#compare['output'] = output
predict_lab = []
predict_name = []
actual_name = []
for i in range(output.shape[0]):
    label = np.argmax(output[i])
    predict_lab.append(label)
    predict_name.append(mc_labels[label])
    actual_name.append(mc_labels[Y_val[i]])
compare['predicted_label'] = predict_lab
compare['actual_label'] = Y_val
compare['predicted_name'] = predict_name
compare['actual_name'] = actual_name

In [35]:
print(predict_lab.count(0) / len(predict_lab))
print(np.count_nonzero(Y_val == 0) / Y_val.shape[0])
print(np.count_nonzero(Y_train == 0) / Y_train.shape[0])
print(np.count_nonzero(Y_val == 0) + np.count_nonzero(Y_train == 0))

0.04070036789753372
0.0347473921666591
0.034876080604381465
115095


In [36]:
compare.sample(n=50)

,predicted_label,actual_label,predicted_name,actual_name
406354,11,11,DiPhotonJetsBox,DiPhotonJetsBox
121627,11,11,DiPhotonJetsBox,DiPhotonJetsBox
440676,11,4,DiPhotonJetsBox,VBFHToGG
137920,11,11,DiPhotonJetsBox,DiPhotonJetsBox
370265,11,11,DiPhotonJetsBox,DiPhotonJetsBox
584619,11,2,DiPhotonJetsBox,VHToGG
411196,11,11,DiPhotonJetsBox,DiPhotonJetsBox
228474,11,11,DiPhotonJetsBox,DiPhotonJetsBox
625516,11,10,DiPhotonJetsBox,GJet_BigPt
114924,11,11,DiPhotonJetsBox,DiPhotonJetsBox


In [ ]:
# following: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

n_classes = Y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score.ravel())


plt.figure()


In [34]:
print(Y_train_dummy)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [35]:
print(model.predict(X_train))

[[3.2191610e-01 1.0798335e-01 6.5229744e-02 ... 4.2139739e-02
  1.2250453e-02 4.8553377e-02]
 [8.0013245e-01 1.0684222e-02 4.4399798e-03 ... 2.8002560e-03
  4.7476411e-02 6.4437091e-03]
 [7.5319552e-01 4.7617555e-03 1.9856989e-03 ... 1.5153289e-03
  7.4679196e-02 3.9824843e-03]
 ...
 [8.0017841e-01 1.2610495e-02 5.7884753e-03 ... 8.4994137e-03
  1.2677610e-02 5.8728456e-04]
 [7.8339016e-01 6.5042377e-03 2.9864609e-03 ... 1.7688870e-03
  4.5274645e-02 2.0082593e-03]
 [7.7708590e-01 4.1216016e-03 1.5176535e-03 ... 1.3048649e-03
  8.0464065e-02 4.5357943e-03]]


In [36]:
y_train_tmp = model.predict(X_train)
predictions_train_tmp = [round(value[0]) for value in y_train_tmp]
train_accuracy = accuracy_score(Y_train_dummy[:, 0], predictions_train_tmp)
print("Training Accuracy: %.2f%%" % (train_accuracy * 100.0))

y_pred_tmp = model.predict(X_val)
predictions_tmp = [round(value[0]) for value in y_pred_tmp]
accuracy = accuracy_score(Y_val, predictions_tmp)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Training Accuracy: 77.94%


In [16]:
def multi_DNN(n_hidden = 1, first_neuron = 100, n_neurons = 64, dropout_rate = 0.4, activation_func = 'sigmoid', lr= 0.00002):
    model = Sequential()
    model.add(Dense(first_neuron, input_dim=len(training_vars), activation='relu'))
    model.add(Dropout(dropout_rate))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation = activation_func))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation = 'sigmoid'))
    opt = Adam(learning_rate = lr)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [37]:
# estimator = KerasClassifier(build_fn=multi_DNN, epochs=200, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, dummy_y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Model Optimization

In [ ]:
space  = [Integer(1, 5, name='n_hidden'),
          Integer(10, 128, name='first_neuron'),
          Integer(10, 128, name = 'n_neurons'),
          Real(0.01,0.9,name='dropout_rate'),
          Categorical(['relu', 'sigmoid', 'softmax']),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate')
         ]

def multi_DNN(n_hidden = 1, first_neuron = 100, n_neurons = 64, dropout_rate = 0.4, activation_func = 'sigmoid', lr= 0.00002):
    model = Sequential()
    model.add(Dense(first_neuron, input_dim=len(training_vars), activation='relu'))
    model.add(Dropout(dropout_rate))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation = activation_func))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation = 'sigmoid'))
    opt = Adam(learning_rate = lr)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# @use_named_args(space)
# def objective(**X):
#     print("New configuration: {}".format(X))
#     model_tmp = multi_DNN(X['n_hidden'], X['first_neuron'], X['n_neurons'], X['dropout_rate'], X['activation_func'], X['lr'])
#     print (model_tmp)
#     estimator_tmp = KerasClassifier(build_fn=model_temp, epochs=200, batch_size=5, verbose=0)
#     kfold = KFold(n_splits=10, shuffle=True)
#     results = cross_val_score(estimator, X, dummy_y, cv=kfold)
#     model_tmp.fit(x_train, y_train, epochs = 50, verbose = 1, validation_split = 0.2,  callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min'),
#                             ModelCheckpoint(filepath='Models/optimize_trial/'+names[i]+'_opt_mass_model.h5', verbose=0)])
    
#     y_train_tmp = model_tmp.predict(x_train)
#     predictions_train_tmp = [round(value[0]) for value in y_train_tmp]
#     train_accuracy = accuracy_score(y_train, predictions_train_tmp)
#     print("Training Accuracy: %.2f%%" % (train_accuracy * 100.0))
    
#     y_pred_tmp = model_tmp.predict(x_test)
#     predictions_tmp = [round(value[0]) for value in y_pred_tmp]
#     accuracy = accuracy_score(y_test, predictions_tmp)
#     print("Accuracy: %.2f%%" % (accuracy * 100.0))
    
#     return -accuracy

In [ ]:
estimator = KerasClassifier(build_fn=multi_DNN, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))